In [4]:
from ipywidgets import widgets, interact
from IPython.display import display
%matplotlib inline
try:
    import seaborn as sbn
except (ModuleNotFoundError, ImportError):
    sbn = None
import matplotlib.pyplot as plt
import numpy as np 
from IPython.core.pylabtools import figsize
figsize(12, 10)
if sbn:
    sbn.set_context("talk", font_scale=1)


In [5]:
from rescure_sim import (mol_heat_rxn, n_coal_rxn,
                    n_crosslinks,
                    T)

ImportError: cannot import name 'mol_heat_rxn' from 'test' (c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\test\__init__.py)

In [10]:
# Constant Input Parameters

Ea = [128, 90, 100, 142] # Activation Energy, KJ/mol
A = [2.34e10, 2.0e10, 2.1e10, 1.8e10] # Pre-exponential Factor
R = 8.314 # Universal Gas Constant, KJ/mol.k

# Assign the values in the Ea list to identifiable variable names
# where pp = phenol-phenol and cp = coal-phenol
# where pp_oo = phenol-phenol_ortho-ortho, pp_op = phenol-phenol_ortho-para, pp_pp = phenol-phenol_para-para

Ea_pp_oo, Ea_pp_op, Ea_pp_pp, Ea_cp = Ea  # Activation Energy for each reaction type
A_pp_oo, A_pp_op, A_pp_pp, A_cp = A # Pre-exponential Factor for each reaction type

print(Ea_pp_oo, Ea_pp_op, Ea_pp_pp, Ea_cp)

# Variable Input Parameters
T = np.linspace(300, 500, 100) # Temperature, K

# Calculating the rate constants for each reaction type
k_pp_oo = A_pp_oo * np.exp(-Ea_pp_oo/(R*T))
k_pp_op = A_pp_op * np.exp(-Ea_pp_op/(R*T))
k_pp_pp = A_pp_pp * np.exp(-Ea_pp_pp/(R*T))
k_cp = A_cp * np.exp(-Ea_cp/(R*T))

Heat_of_rxn = [] # Heat of Reaction 


128 90 100 142


In [ ]:
@interact(p_phenol=widgets.FloatSlider(value=2.3, min=2.3, max=22, step=0.5), 
          o_phenol=widgets.FloatSlider(value=24, min=10, max=100, step=5), 
          coal=widgets.FloatSlider(value=0, min=0, max=100, step=5),
          void=widgets.FloatSlider(value=0, min=0, max=100, step=5))

def reaction(p_phenol, o_phenol, coal, void):
    
    """Calculates the degree of crosslinking and temperature change 
    based on the input parameters using functions from the KMC_model module"""
    
    # Calculate the degree of crosslinking based on the input parameters
    degree_cross = Degree_of_cross(p_phenol, o_phenol, coal, void)
    
    # Calculate the temperature change based on the degree of crosslinking
    delta_T = temp_change(degree_cross)
    
    # Calculate the heat of reaction for each reaction type
    heat_rxn_pp_oo = Heat_of_rxn(k_pp_oo, delta_T)
    heat_rxn_pp_op = Heat_of_rxn(k_pp_op, delta_T)       
    heat_rxn_pp_pp = Heat_of_rxn(k_pp_pp, delta_T)
    heat_rxn_cp = Heat_of_rxn(k_cp, delta_T)
    
    """Appends the calculated heat of reaction values to the Heat_of_rxn list for later plotting"""
    
    Heat_of_rxn.append([heat_rxn_pp_oo, heat_rxn_pp_op, heat_rxn_pp_pp, heat_rxn_cp])
    
    # Plot the sensitivity analysis
    plot_sensitivity_analysis(T, [k_pp_oo, k_pp_op, k_pp_pp, k_cp],
                              degree_cross, delta_T, Heat_of_rxn)

# Define the sensitivity analysis plot function
def plot_sensitivity_analysis(temperature, rate_constants, degree_cross, delta_T, heat_of_rxn):
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))

    # Plot Rate Constants
    ax1.set_xlabel('Temperature (K)')
    ax1.set_ylabel('Rate Constants')
    ax1.plot(temperature, rate_constants[0], label='k_pp_oo')
    ax1.plot(temperature, rate_constants[1], label='k_pp_op')
    ax1.plot(temperature, rate_constants[2], label='k_pp_pp')
    ax1.plot(temperature, rate_constants[3], label='k_cp')
    ax1.legend(loc='upper left')

    # Plot Degree of Crosslinking
    ax2.set_xlabel('Temperature (K)')
    ax2.set_ylabel('Degree of Crosslinking')
    ax2.plot(temperature, degree_cross, label='Degree of Crosslinking', color='tab:orange')
    ax2.legend(loc='upper left')

    # Plot Temperature Change
    ax3.set_xlabel('Temperature (K)')
    ax3.set_ylabel('Temperature Change')    
    ax3.plot(temperature, delta_T, label='Temperature Change', color='tab:green')
    ax3.legend(loc='upper left')

    fig.tight_layout()
    plt.show()